# AI-Med: Fine-Tuning Llama 3.2 1B (Edición Ultra-Rápida)
Este notebook entrena el modelo de 1 billón de parámetros para máxima velocidad en local.

In [ ]:
%%capture
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes datasets

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-1B-Instruct-bnb-4bit",
    max_seq_length = max_seq_length,
    load_in_4bit = load_in_4bit,
)

model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
)

In [ ]:
battlecard_prompt = """Eres el Agente Generador Experto de AI-Med Learning.
Extrae la información médica del [Contexto] y conviértela en una [BattleCard].

### Contexto Científico:
{}

### BattleCard (Formato Estricto):
{}"""

EOS_TOKEN = tokenizer.eos_token

def formatting_prompts_func(examples):
    contextos = examples["contexto_mcp"]
    salidas   = examples["casos_generados"]
    texts = []
    for contexto, salida in zip(contextos, salidas):
        text = battlecard_prompt.format(contexto, salida) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }

from datasets import load_dataset
dataset = load_dataset("json", data_files="dataset_casos_clinicos.jsonl", split="train")
dataset = dataset.map(formatting_prompts_func, batched = True)

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model, tokenizer = tokenizer, train_dataset = dataset,
    dataset_text_field = "text", max_seq_length = max_seq_length,
    args = TrainingArguments(
        per_device_train_batch_size = 4,
        gradient_accumulation_steps = 4,
        warmup_steps = 5, max_steps = 150,
        learning_rate = 3e-4, fp16 = True, logging_steps = 1,
        optim = "adamw_8bit", output_dir = "outputs",
    ),
)
trainer.train()

In [ ]:
model.save_pretrained_gguf("Llama_3.2_1B_AI_Med", tokenizer, quantization_method = "q4_k_m")

from google.colab import drive
import shutil
import os

# Montar Google Drive
drive.mount('/content/drive')

try:
    gguf_file = [f for f in os.listdir("Llama_3.2_1B_AI_Med") if f.endswith(".gguf")][0]
    source_path = os.path.join("Llama_3.2_1B_AI_Med", gguf_file)
    destination_path = f"/content/drive/MyDrive/{gguf_file}"
    
    print(f"Copiando {source_path} a tu Google Drive... (Esto tomará unos minutos)")
    shutil.copy2(source_path, destination_path)
    print("¡Copia completada! Ve a drive.google.com y descarga el archivo .gguf a tu Mac M3.")
except Exception as e:
    print(f"Error copiando a drive: {e}")